In [9]:
import geopandas as gpd
import numpy as np
from ag_vision.drone import utils as du
from ag_vision.constants import paths as pth
from ag_vision.data_io import databricks_io as dio

In [ ]:
save_volume = "/Volumes/ue1_prod_catalog_119738067017277/dan_test/drone_data"

In [ ]:
base_folder = "/Volumes/ciat-rice-pheno-i-input-data/default/ciat-rice-data/Rice/Hoja Blanca/RHBV_2024_B/"
ortho_path = f"{base_folder}/Multispectral/2024_11_28_09_00/MS.tif"
dem_path = f"{base_folder}/Multispectral/2024_11_28_09_00/DEM.tif"

In [4]:
task = 'drone_phenotyping'
collection_date = '11/28/2024'
flight_name = 'flight_000'

trial = 'RHBV'

site = 'CIAT-Cali'
field = 'Hoja Blanca'
location = 'B'

year = 2024
country = 'col'
crop = 'rice'
time_of_year = '11'

In [7]:
season_code = pth.season_code(year=year,
                              country=country,
                              crop=crop,
                              time_of_year=time_of_year)

loc_path = pth.location_path(project='dan_test',
                             site=site,
                             trial=trial,
                             season=season_code,
                             field=field,
                             location=location)

mission_path = pth.drone_mission_dir(location_path=loc_path,
                                 mission_name='test')

print(mission_path)


dan_test/ciat-cali/rhbv/2024:col:rice:11/hoja blanca/b/drone/test


In [ ]:
plot_gdf = gpd.read_file(f"{base_folder}/Field data/05_ROI/PLOTS.shp")
plot_gdf.plot()

In [ ]:
# read in both the ortho data and the dem data so that we can crop the images down to the plot area.
# Once we have the images at the plot level we can the calculate things like aver height of the plot or metrics like NDVI on the multispec data (MS)
ortho_img, ortho_transform, ortho_nodata, ortho_tif_crs = du.read_geotiff(ortho_path)
dem_img, dem_transform, dem_nodata, dem_tif_crs = du.read_geotiff(dem_path)

In [ ]:
# Need to make sure the crs of the plot data match that of the raster data. If not the join between the two will not be corred.
ortho_plots_gdf = plot_gdf.to_crs(ortho_tif_crs)

In [ ]:
row = ortho_plots_gdf.loc[1, :]
print(row)
dem_mask = du.generate_mask_from_polygon(polygon=row.geometry,
                                      img_transform=dem_transform,
                                      img_x=dem_img.shape[1],
                                      img_y=dem_img.shape[2])

ortho_mask = du.generate_mask_from_polygon(polygon=row.geometry,
                                        img_transform=ortho_transform,
                                        img_x=ortho_img.shape[1],
                                        img_y=ortho_img.shape[2])

In [ ]:
dem_masked_img = du.apply_mask_to_image(image=dem_img,
                                         mask=dem_mask,
                                         fill_value=np.nan)

ortho_masked_img = du.apply_mask_to_image(image=ortho_img,
                                           mask=ortho_mask,
                                           fill_value=np.nan)

In [ ]:
cropped_dem = du.crop_image_by_mask(image=dem_masked_img,
                                     mask=dem_mask,
                                     channel_first=True)

cropped_ortho = du.crop_image_by_mask(image=ortho_masked_img,
                                       mask=ortho_mask,
                                       channel_first=True)

In [ ]:
plt.imshow(cropped_dem[0,:,:])

plt.show()

In [ ]:
dio.save_tif_to_databricks(img=cropped_dem,
                       file_name=f"{save_dir}/a/b/bv/dem_test.tif")

In [ ]:
a = dio.read_tiff_from_databricks(file_name="/Volumes/ue1_prod_catalog_119738067017277/dan_test/dem/dem_test.tif")
